# Wildfire Visualization with Plotly + MapLibre

This notebook provides fully programmatic wildfire visualizations using Plotly with MapLibre.

**Advantages:**
- Complete programmatic control over styling
- Free CARTO basemaps (no API key required)
- Interactive plots with hover data
- Density heatmaps for air quality
- Time-series animations (if needed)
- Export to standalone HTML

**No Setup Required** - Uses free CARTO tiles via MapLibre (Plotly 5.24+)

## Setup

In [1]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import json

In [2]:
# Configure map style
# Plotly 5.24+ uses MapLibre with free CARTO tiles
# Available styles:
#   - 'carto-positron': Light theme (default)
#   - 'carto-darkmatter': Dark theme
#   - 'carto-voyager': Colorful theme
#   - 'open-street-map': Classic OpenStreetMap

map_style = 'carto-positron'  # Light theme

print(f'✓ Using MapLibre with {map_style}')
print('  No API key required')
print(f'  Attribution: © CARTO © OpenStreetMap')

✓ Using MapLibre with carto-positron
  No API key required
  Attribution: © CARTO © OpenStreetMap


## Load Data

In [3]:
# Load GeoJSON files from ETL output
data_dir = Path('../data/processed')
fires_gdf = gpd.read_file(data_dir / 'active_fires.geojson')
buffers_gdf = gpd.read_file(data_dir / 'fire_buffers.geojson')

# Convert to regular DataFrames (Plotly works with lat/lon columns)
fires_df = fires_gdf.copy()
fires_df['lon'] = fires_df.geometry.x
fires_df['lat'] = fires_df.geometry.y

# Calculate map center from fire locations
center_lat = fires_df['lat'].mean()
center_lon = fires_df['lon'].mean()

print(f"Loaded {len(fires_df)} fire detections")
print(f"Loaded {len(buffers_gdf)} buffer zones")
print(f"Map center: ({center_lat:.4f}, {center_lon:.4f})")

Loaded 461 fire detections
Loaded 3 buffer zones
Map center: (32.1795, -83.6785)


## 1. Basic Fire Points Map

Simple scatter plot showing all fire detections colored by risk category.

In [4]:
fig = px.scatter_map(
    fires_df,
    lat='lat',
    lon='lon',
    color='risk_category',
    color_discrete_map={'Low': 'yellow', 'Moderate': 'orange', 'High': 'red'},
    hover_data=['risk_score', 'confidence', 'bright_ti4', 'frp', 'acq_date'],
    zoom=6,
    center={'lat': center_lat, 'lon': center_lon},
    title='Active Fire Detections by Risk Category',
    map_style=map_style,
)

fig.update_layout(height=600)
fig.show()

# Export to HTML
fig.write_html('../data/processed/fires_basic_map.html')
print('✓ Exported to data/processed/fires_basic_map.html')

ValueError: Value of 'hover_data_2' is not the name of a column in 'data_frame'. Expected one of ['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_ti5', 'frp', 'daynight', 'acq_datetime', 'temperature_c', 'relative_humidity', 'wind_speed_kmh', 'wind_direction_deg', 'precip_probability', 'fire_danger_index', 'red_flag_index', 'aqi', 'aqi_parameter', 'aqi_category', 'aqi_category_number', 'pa_pm25', 'pa_pm25_60min', 'pa_sensor_count', 'pa_avg_distance_km', 'brightness_norm', 'confidence_norm', 'frp_norm', 'daynight_norm', 'humidity_norm', 'wind_norm', 'precip_norm', 'fire_danger_norm', 'risk_score', 'risk_category', 'uses_weather_data', 'geometry', 'lon', 'lat'] but received: brightness

## 2. Risk Score Heatmap

Density map weighted by fire risk scores.

In [ ]:
fig = px.density_map(
    fires_df,
    lat='lat',
    lon='lon',
    z='risk_score',  # Weight by risk score
    radius=15,  # Heatmap radius in pixels
    zoom=6,
    center={'lat': center_lat, 'lon': center_lon},
    color_continuous_scale='YlOrRd',  # Yellow -> Orange -> Red
    title='Fire Risk Heatmap (weighted by risk score)',
    map_style=map_style,
)

fig.update_layout(height=600)
fig.show()

fig.write_html('../data/processed/fires_risk_heatmap.html')
print('✓ Exported to data/processed/fires_risk_heatmap.html')

## 3. Fire Intensity (Brightness Temperature)

Points sized by brightness temperature with color gradient.

In [ ]:
fig = px.scatter_map(
    fires_df,
    lat='lat',
    lon='lon',
    size='bright_ti4',
    color='bright_ti4',
    color_continuous_scale='Hot',
    hover_data=['bright_ti4', 'bright_ti5', 'frp', 'confidence', 'acq_date'],
    zoom=6,
    center={'lat': center_lat, 'lon': center_lon},
    title='Fire Brightness Temperature - Band I4 (K)',
    map_style=map_style,
    size_max=15,
)

fig.update_layout(height=600)
fig.show()

fig.write_html('../data/processed/fires_brightness.html')
print('✓ Exported to data/processed/fires_brightness.html')

## 4. Risk Buffer Zones

Choropleth map showing dissolved risk buffer zones.

In [ ]:
# Convert GeoDataFrame to GeoJSON for Plotly
buffers_geojson = json.loads(buffers_gdf.to_json())

fig = px.choropleth_map(
    buffers_gdf,
    geojson=buffers_geojson,
    color='risk_category',
    locations=buffers_gdf.index,
    color_discrete_map={'Low': 'yellow', 'Moderate': 'orange', 'High': 'red'},
    hover_data=['risk_category'],
    zoom=6,
    center={'lat': center_lat, 'lon': center_lon},
    title='Fire Risk Buffer Zones',
    map_style=map_style,
    opacity=0.5,
)

fig.update_layout(height=600)
fig.show()

fig.write_html('../data/processed/fires_buffers.html')
print('✓ Exported to data/processed/fires_buffers.html')

## 5. Combined: Fires + Buffer Zones

Overlay fire points on top of buffer zones.

In [ ]:
# Create choropleth for buffers
fig = go.Figure()

# Add buffer zones
for idx, row in buffers_gdf.iterrows():
    color_map = {'Low': 'rgba(255,255,0,0.3)', 'Moderate': 'rgba(255,165,0,0.3)', 'High': 'rgba(255,0,0,0.3)'}
    
    if row.geometry.geom_type == 'Polygon':
        coords = list(row.geometry.exterior.coords)
    elif row.geometry.geom_type == 'MultiPolygon':
        # Take the largest polygon
        coords = list(max(row.geometry.geoms, key=lambda p: p.area).exterior.coords)
    else:
        continue
    
    lons, lats = zip(*coords)
    
    fig.add_trace(go.Scattermap(
        lon=lons,
        lat=lats,
        mode='lines',
        fill='toself',
        fillcolor=color_map.get(row['risk_category'], 'rgba(128,128,128,0.3)'),
        line=dict(width=1, color='white'),
        name=f"{row['risk_category']} Risk Zone",
        showlegend=True,
        hovertemplate=f"Risk: {row['risk_category']}<extra></extra>",
    ))

# Add fire points
color_map_points = {'Low': 'yellow', 'Moderate': 'orange', 'High': 'red'}
for category in ['Low', 'Moderate', 'High']:
    subset = fires_df[fires_df['risk_category'] == category]
    if len(subset) > 0:
        fig.add_trace(go.Scattermap(
            lon=subset['lon'],
            lat=subset['lat'],
            mode='markers',
            marker=dict(size=8, color=color_map_points[category]),
            name=f"{category} Risk Fires",
            hovertemplate='<b>Risk:</b> %{text}<br><b>Score:</b> %{customdata[0]:.1f}<extra></extra>',
            text=[category] * len(subset),
            customdata=subset[['risk_score']].values,
        ))

fig.update_layout(
    title='Fire Detections with Risk Buffer Zones',
    map=dict(
        style=map_style,
        center=dict(lat=center_lat, lon=center_lon),
        zoom=6,
    ),
    height=600,
)

fig.show()

fig.write_html('../data/processed/fires_combined.html')
print('✓ Exported to data/processed/fires_combined.html')

## 6. AirNow AQI Data (if available)

Shows fires enriched with EPA AirNow air quality data.

In [ ]:
# Filter fires that have AQI data
fires_aqi = fires_df[fires_df['aqi'].notna()].copy()

if len(fires_aqi) > 0:
    # Use density map for AQI heatmap visualization
    fig = px.density_map(
        fires_aqi,
        lat='lat',
        lon='lon',
        z='aqi',  # Weight by AQI value
        radius=15,  # Heatmap radius in pixels
        zoom=6,
        center={'lat': center_lat, 'lon': center_lon},
        color_continuous_scale='YlOrRd',
        title='Air Quality Index (AQI) Density Heatmap',
        map_style=map_style,
        hover_data=['aqi', 'aqi_category', 'aqi_parameter'],
    )
    
    # Fix layout to match other maps
    fig.update_layout(
        height=600,
        margin=dict(l=0, r=0, t=30, b=0),
        autosize=True,
    )
    fig.show()
    
    fig.write_html('../data/processed/fires_aqi.html')
    print(f'✓ Exported to data/processed/fires_aqi.html ({len(fires_aqi)} fires with AQI data)')
else:
    print('⚠ No AQI data available. Run ETL with --aqi flag to enrich with AirNow data.')

## 7. PurpleAir PM2.5 Heatmap (if available)

Density heatmap of PM2.5 concentrations from nearby PurpleAir sensors.

In [ ]:
# Filter fires that have PurpleAir data
fires_pa = fires_df[fires_df['pa_pm25'].notna()].copy()

if len(fires_pa) > 0:
    fig = px.density_map(
        fires_pa,
        lat='lat',
        lon='lon',
        z='pa_pm25',  # Weight by PM2.5 concentration
        radius=15,  # Heatmap radius in pixels
        zoom=6,
        center={'lat': center_lat, 'lon': center_lon},
        color_continuous_scale='YlOrRd',
        title='PM2.5 Smoke Density Heatmap (PurpleAir)',
        map_style=map_style,
        hover_data=['pa_pm25', 'pa_sensor_count'],
    )
    
    # Fix layout to match other maps
    fig.update_layout(
        height=600,
        margin=dict(l=0, r=0, t=30, b=0),
        autosize=True,
    )
    fig.show()
    
    fig.write_html('../data/processed/fires_purpleair_heatmap.html')
    print(f'✓ Exported to data/processed/fires_purpleair_heatmap.html ({len(fires_pa)} fires with PM2.5 data)')
else:
    print('⚠ No PurpleAir data available. Run ETL with --purpleair flag to enrich with sensor data.')

## Summary

All visualizations have been exported to `data/processed/` as standalone HTML files:

1. `fires_basic_map.html` - Basic fire points by risk category
2. `fires_risk_heatmap.html` - Risk score density heatmap
3. `fires_brightness.html` - Fire brightness/intensity map
4. `fires_buffers.html` - Risk buffer zones
5. `fires_combined.html` - Fires + buffer zones overlay
6. `fires_aqi.html` - AirNow AQI data (if available)
7. `fires_purpleair_heatmap.html` - PurpleAir PM2.5 heatmap (if available)

All maps use free MapLibre rendering with CARTO basemaps - no API keys required!